In [14]:
import pandas as pd
import numpy as np

In [74]:
#def rename_columns(path = "data/cifar10_cons.csv"):
path = "data/exp2/imdb_pos_emb_test.csv"
df = pd.read_csv(path)
df = df.drop(["Step"], axis='columns')
min_max = [col for col in list(df.columns) if "MIN" in col or "MAX" in col]
df = df.drop(min_max, axis='columns')

not_all = [col for col in list(df.columns) if "\'first\'" in col or "\'everyother\'"  in col]
print("not all", not_all)
df = df.drop(not_all, axis='columns')
df = df.dropna(axis = "columns")
df = df.rename(columns={name: name.lower()[:-11] for name in list(df.columns)})

imdb = True
vars = ["mamba", "diag","10000", "1024", "true"]
bonus_vars = ["b", "c", "dt", "x"]
final_vars = ["no_emb", "imdb","acc"]
all_vars = vars + bonus_vars + final_vars
num_vars = len(all_vars)
old_names = list(df.columns)


binary_data = np.zeros((len(df.columns), num_vars))
for name_dix, name in enumerate(old_names):
    for var_idx, var in enumerate(vars):
        if var in name:
            binary_data[name_dix, var_idx] = 1.
        
        # get b,c,dt,x for s6
        if "s6" in name and "[" in name:
            subset = name.split(" ")[-1]
            for var_idx2, var in enumerate(bonus_vars):
                if var in subset:
                    binary_data[name_dix, len(vars) + var_idx2] = 1.
        
        # get acc and dataset
        acc = df[name].values[-1]
        binary_data[name_dix, -1] = acc
        imdb_ = imdb
        if imdb_:
            binary_data[name_dix, -2] = 1
           
        # no embs at all
        if len(name.split(" ")) < 3:
            binary_data[name_dix, -3] = 1
            
            #print(name)
            
          
            
    # old_name = name
    # name = name.lower()
    # new_name = ""
    # if "classicmodel" in name:
    #     new_name += "Simple-"
    # elif "mamba" in name:
    #     new_name += "Mamba-"
    # else:
    #     raise NameError
    # 
    # if "diag" in name:
    #     new_name += "s4d"
    # elif "dplr" in name:
    #     new_name += "s4"
    # elif "s6" in name:
    #     new_name += "s6"
    # else:
    #     raise NameError
    # 
    # df = df.rename(columns={old_name: new_name})
#df = df.reindex(sorted(df.columns, reverse=True), axis=1)
#df.file_name = path
    #return df

#df = rename_columns()

not all []


In [75]:
print(list(zip(all_vars, binary_data.mean(axis=0))))

[('mamba', 0.7142857142857143), ('diag', 0.32142857142857145), ('10000', 0.4642857142857143), ('1024', 0.6428571428571429), ('true', 0.39285714285714285), ('b', 0.42857142857142855), ('c', 0.42857142857142855), ('dt', 0.35714285714285715), ('x', 0.2857142857142857), ('no_emb', 0.03571428571428571), ('imdb', 1.0), ('acc', 0.7929699949114665)]


In [76]:
binary_data[:,:-1]

array([[1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
       [1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1.],
       [1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.],
       [1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1.],
       [1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
       [1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1.],
       [1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1.],
       [1., 0., 1., 0., 0., 0., 0., 1., 0., 0.

In [ ]:
df.datafram

In [79]:
import statsmodels.api as sm
y = binary_data[:,-1]
x = binary_data[:,:-2]
x = sm.add_constant(x, has_constant="raise")
mod = sm.OLS(y,x)
fit = mod.fit()
print(fit.params)
fit.summary(xname=["cons"]+all_vars[:-2])


[ 0.65708796  0.15340161 -0.00648068  0.02509435  0.05045993 -0.01342887
 -0.00391905 -0.00391905 -0.0083542  -0.0193485   0.04059043]


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                     46.41
Date:                Mon, 29 Apr 2024   Prob (F-statistic):           1.24e-10
Time:                        22:22:35   Log-Likelihood:                 77.500
No. Observations:                  28   AIC:                            -135.0
Df Residuals:                      18   BIC:                            -121.7
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cons           0.6571      0.024     27.161      0.000       0.606       0.708
mamba          0.1534      0.021      7.371      0.000       0.110       0.197
diag          -0.0065      0.024     -0.269      0.791      -0.057       0.044
10000          0.0251      0.009      2.764      0.013       0.006       0.044
1024           0.0505      0.010      5.205      0.000       0.030       0.071
true          -0.0134      0.008     -1.633      0.120      -0.031       0.004
b             -0.0039      0.005     -0.794      0.437      -0.014       0.006
c             -0.0039      0.005     -0.794      0.437      -0.014       0.006
dt            -0.0084      0.012     -0.697      0.495      -0.034       0.017
x             -0.0193      0.011     -1.700      0.106      -0.043       0.005
no_emb         0.0406      0.023      1.766      0.094      -0.008       0.089
==============================================================================
Omnibus:                        0.139   Durbin-Watson:                   2.105
Prob(Omnibus):                  0.933   Jarque-Bera (JB):                0.336
Skew:                          -0.110   Prob(JB):                        0.845
Kurtosis:                       2.510   Cond. No.                     2.57e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.38e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""